# Dependencies and notebook settings

In [90]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.api as sm
from tqdm import tqdm
from IPython.core.display import display, HTML
import ngboost
import talib
from sklearn.tree import DecisionTreeRegressor

display(HTML("<style>.container { width:90% !important; }</style>"))
plt.style.use("ggplot")
mpl.rcParams["figure.dpi"] = 100
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)

# Data import and preparation

In [91]:
df = pd.read_csv(
    "../dataset/spx.csv",
    parse_dates=["Date"],
    names=["Date", "Open", "High", "Low", "Close", "Volume"],
    header=0,
    index_col="Date")
df = df[df.index < "2020-10-01"]
df["rr"] = (np.log(df.Close) - np.log(df.Close.shift(1))) * 100
df.dropna(inplace=True)

# Feature engineering

## Time variables

In [92]:
df["day_of_week"] = df.index.dayofweek
df["day_of_year"] = df.index.dayofyear
df["week"] = df.index.week
df["quarter"] = df.index.quarter

## Stationarization 

In [93]:
df["Open_stationary"] = df["Open"].diff()
df["High_stationary"]= df["High"].diff()
df["Low_stationary"]= df["Low"].diff()
df["Close_stationary"]= df["Close"].diff()

## Intra day relations

In [94]:
df["Close_minus_Open"] = df["Close"] - df["Open"]
df["High_minus_Low"] = df["High"] - df["Low"]

## Shallow technical analysis variables and ARMA proxy 

### Simple statistics 

In [95]:
df["MA"] = talib.MA(df["rr"], timeperiod=2)
df["EMA"] = talib.EMA(df["rr"], timeperiod=2)
df["STD"] = talib.STDDEV(df["rr"], timeperiod=2)
df["MA_2"] = talib.MA(df["rr"], timeperiod=3)
df["EMA_2"] = talib.EMA(df["rr"], timeperiod=3)
df["STD_2"] = talib.STDDEV(df["rr"], timeperiod=3)
df["MA_W"] = talib.MA(df["rr"], timeperiod=6)
df["EMA_W"] = talib.EMA(df["rr"], timeperiod=6)
df["STD_W"] = talib.STDDEV(df["rr"], timeperiod=6)

### Volatility Indicators 

In [96]:
df["ATR"] = talib.ATR(df["High"], df["Low"], df["Close"], timeperiod=7)
df["TRANGE"] = talib.TRANGE(df["High"], df["Low"], df["Close"])

### Volume Indicators 

In [97]:
df["OBV"] = talib.OBV(df["Close"],df["Volume"])
df["ADOSC"] = talib.ADOSC(df["High"], df["Low"], df["Close"], df["Volume"])

## Dataset cleaning 

In [98]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'rr', 'day_of_week',
       'day_of_year', 'week', 'quarter', 'Open_stationary', 'High_stationary',
       'Low_stationary', 'Close_stationary', 'Close_minus_Open',
       'High_minus_Low', 'MA', 'EMA', 'STD', 'MA_2', 'EMA_2', 'STD_2', 'MA_W',
       'EMA_W', 'STD_W', 'ATR', 'TRANGE', 'OBV', 'ADOSC'],
      dtype='object')

In [99]:
df = df[['rr', 'day_of_week','day_of_year', 'week', 'quarter', 
         'Volume','Open_stationary', 'High_stationary',
       'Low_stationary', 'Close_stationary', 'Close_minus_Open',
       'High_minus_Low', 'MA', 'EMA', 'STD', 'MA_2', 'EMA_2', 'STD_2', 'MA_W',
       'EMA_W', 'STD_W', 'ATR', 'TRANGE', 'OBV', 'ADOSC']]

## Variables lagging/shifting

In [100]:
to_lag = ['rr', 'Volume', 'Open_stationary', 'High_stationary', 'Low_stationary','Close_stationary', 
          'Close_minus_Open', 'High_minus_Low', 'MA', 'EMA',  'STD', 'MA_2', 'EMA_2', 'STD_2', 'MA_W', 'EMA_W', 
          'STD_W', 'ATR', 'TRANGE', 'OBV', 'ADOSC']

for i in to_lag:
    for j in range(1,4):
        col_name = i + "_L" + str(j)
        df[col_name] = df[i].shift(j)
    if i != "rr":
        df.drop(columns = [i], inplace=True)

In [101]:
df.shape

(7664, 68)

In [13]:
features = set(df.columns)
features.remove("rr")

# Feature selection using tree-based feature selection

To understand my feature selection procedure it is necessary to know that: "NGBoost does provide methods to interpret models fit with regression tree base learners. Since each parameter in the distribution is fit by a separate sequence of learners, there will be multiple model interpretation results, one for each parameter. The default distribution used is Normal so the following example shows results for the loc and scale parameters." [source](https://stanfordmlgroup.github.io/ngboost/3-interpretation.html)

Taking above into consideration I will assume that my default NGBoost model (during next modeling steps I will tune this parameters but now I will fix them) model consists of: 
* Normal distribution as the output distribution
* 3-depth Decision Tree as the base learner
* 500 iterations as the number of boosting iterations
* 0.01 learning rate
* negative log likelihood score as scoring function

My procedure is as follows: for each training-validation period I will compute feature importance for both distribution parameters (loc, scale) and then their average value. Then I will compute overall mean value based on output from 3 available training-validation periods. Finally I decided expertly I will choose final features based on threshold for final average feature importance which is equal to 25.  

In [62]:
starts = ["2006-01-12", "2008-01-15", "2014-01-13"]

## Training-validation period 1

In [15]:
loc_list = list()
scale_list = list()

start = starts[0]
df_tmp = df.loc[start:].head(252 * 4).copy()

for i in tqdm(range(0, 252)):
    train = df_tmp.iloc[i : i + 252 * 3].copy()
    ngb = ngboost.NGBRegressor(
        Dist=ngboost.distns.Normal,
        Score=ngboost.scores.LogScore,
        Base=DecisionTreeRegressor(criterion="friedman_mse", max_depth=3),
        n_estimators=500,
        learning_rate=0.01,
        minibatch_frac=1.0,
        col_sample=1.0,
        verbose=False,
        verbose_eval=500,
        tol=0.0001,
        random_state=2021)
    ngb.fit(train[features], train.rr)
    
    feature_importance_loc = ngb.feature_importances_[0]
    feature_importance_scale = ngb.feature_importances_[1]
    df_loc = pd.DataFrame({'feature':list(features),
                           'importance':feature_importance_loc}).sort_values('importance',ascending=False)
    df_scale = pd.DataFrame({'feature':list(features),
                           'importance':feature_importance_scale}).sort_values('importance',ascending=False)    
    df_loc["index"] = np.arange(1,68)
    df_loc.drop(columns=["importance"], inplace=True)
    df_scale["index"] = np.arange(1,68)
    df_scale.drop(columns=["importance"], inplace=True)
    loc_list.append(df_loc)
    scale_list.append(df_scale)

loc_df1 = pd.concat(loc_list)
scale_df1 = pd.concat(scale_list)

100%|██████████████████████████████████████████████████████████████████████████████| 252/252 [1:05:49<00:00, 15.67s/it]


In [44]:
loc_df1_gb = loc_df1.groupby("feature").mean().sort_values("index").reset_index()
scale_df1_gb = scale_df1.groupby("feature").mean().sort_values("index").reset_index()
df1_gb = pd.merge(loc_df1_gb, scale_df1_gb, on="feature")
df1_gb["mean_indexes"] = (df1_gb.index_x + df1_gb.index_y)/2
df1_gb.sort_values("mean_indexes", inplace=True)

In [49]:
df1_gb.head(10)

,feature,index_x,index_y,mean_indexes
4,Volume_L1,5.055556,6.579365,5.817460
3,MA_2_L3,4.892857,20.801587,12.847222
5,OBV_L1,6.952381,19.634921,13.293651
1,EMA_W_L2,3.349206,25.543651,14.446429
16,Volume_L2,21.317460,8.960317,15.138889
13,STD_L3,15.408730,14.932540,15.170635
19,EMA_L1,24.579365,6.023810,15.301587
15,STD_W_L2,18.206349,12.611111,15.408730
14,Volume_L3,15.829365,16.932540,16.380952
0,Open_stationary_L3,2.456349,33.281746,17.869048


## Training-validation period 2

In [16]:
loc_list = list()
scale_list = list()

start = starts[1]
df_tmp = df.loc[start:].head(252 * 4).copy()

for i in tqdm(range(0, 252)):
    train = df_tmp.iloc[i : i + 252 * 3].copy()
    ngb = ngboost.NGBRegressor(
        Dist=ngboost.distns.Normal,
        Score=ngboost.scores.LogScore,
        Base=DecisionTreeRegressor(criterion="friedman_mse", max_depth=3),
        n_estimators=500,
        learning_rate=0.01,
        minibatch_frac=1.0,
        col_sample=1.0,
        verbose=False,
        verbose_eval=500,
        tol=0.0001,
        random_state=2021)
    ngb.fit(train[features], train.rr)
    
    feature_importance_loc = ngb.feature_importances_[0]
    feature_importance_scale = ngb.feature_importances_[1]
    df_loc = pd.DataFrame({'feature':list(features),
                           'importance':feature_importance_loc}).sort_values('importance',ascending=False)
    df_scale = pd.DataFrame({'feature':list(features),
                           'importance':feature_importance_scale}).sort_values('importance',ascending=False)    
    df_loc["index"] = np.arange(1,68)
    df_loc.drop(columns=["importance"], inplace=True)
    df_scale["index"] = np.arange(1,68)
    df_scale.drop(columns=["importance"], inplace=True)
    loc_list.append(df_loc)
    scale_list.append(df_scale)

loc_df2 = pd.concat(loc_list)
scale_df2 = pd.concat(scale_list)

100%|██████████████████████████████████████████████████████████████████████████████| 252/252 [1:04:25<00:00, 15.34s/it]


In [45]:
loc_df2_gb = loc_df2.groupby("feature").mean().sort_values("index").reset_index()
scale_df2_gb = scale_df2.groupby("feature").mean().sort_values("index").reset_index()
df2_gb = pd.merge(loc_df2_gb, scale_df2_gb, on="feature")
df2_gb["mean_indexes"] = (df2_gb.index_x + df2_gb.index_y)/2
df2_gb.sort_values("mean_indexes", inplace=True)

In [50]:
df2_gb.head(10)

,feature,index_x,index_y,mean_indexes
3,Volume_L1,8.476190,14.075397,11.275794
6,MA_W_L3,10.388889,12.329365,11.359127
12,TRANGE_L3,18.480159,9.337302,13.908730
14,EMA_W_L1,19.849206,9.416667,14.632937
11,MA_L1,17.904762,11.694444,14.799603
2,Open_stationary_L3,5.452381,25.849206,15.650794
0,MA_2_L3,3.841270,27.853175,15.847222
7,MA_W_L2,10.928571,22.289683,16.609127
26,High_minus_Low_L3,28.996032,4.888889,16.942460
16,MA_2_L2,21.992063,15.003968,18.498016


## Training-validation period 3

In [17]:
loc_list = list()
scale_list = list()

start = starts[2]
df_tmp = df.loc[start:].head(252 * 4).copy()

for i in tqdm(range(0, 252)):
    train = df_tmp.iloc[i : i + 252 * 3].copy()
    ngb = ngboost.NGBRegressor(
        Dist=ngboost.distns.Normal,
        Score=ngboost.scores.LogScore,
        Base=DecisionTreeRegressor(criterion="friedman_mse", max_depth=3),
        n_estimators=500,
        learning_rate=0.01,
        minibatch_frac=1.0,
        col_sample=1.0,
        verbose=False,
        verbose_eval=500,
        tol=0.0001,
        random_state=2021)
    ngb.fit(train[features], train.rr)
    
    feature_importance_loc = ngb.feature_importances_[0]
    feature_importance_scale = ngb.feature_importances_[1]
    df_loc = pd.DataFrame({'feature':list(features),
                           'importance':feature_importance_loc}).sort_values('importance',ascending=False)
    df_scale = pd.DataFrame({'feature':list(features),
                           'importance':feature_importance_scale}).sort_values('importance',ascending=False)    
    df_loc["index"] = np.arange(1,68)
    df_loc.drop(columns=["importance"], inplace=True)
    df_scale["index"] = np.arange(1,68)
    df_scale.drop(columns=["importance"], inplace=True)
    loc_list.append(df_loc)
    scale_list.append(df_scale)

loc_df3 = pd.concat(loc_list)
scale_df3 = pd.concat(scale_list)

100%|██████████████████████████████████████████████████████████████████████████████| 252/252 [1:04:43<00:00, 15.41s/it]


In [46]:
loc_df3_gb = loc_df3.groupby("feature").mean().sort_values("index").reset_index()
scale_df3_gb = scale_df3.groupby("feature").mean().sort_values("index").reset_index()
df3_gb = pd.merge(loc_df3_gb, scale_df3_gb, on="feature")
df3_gb["mean_indexes"] = (df3_gb.index_x + df3_gb.index_y)/3
df3_gb.sort_values("mean_indexes", inplace=True)

In [51]:
df3_gb.head(10)

,feature,index_x,index_y,mean_indexes
1,ADOSC_L1,5.095238,7.357143,4.150794
2,ADOSC_L3,5.567460,8.035714,4.534392
7,Volume_L2,14.099206,8.047619,7.382275
13,OBV_L1,19.071429,3.448413,7.506614
18,Volume_L1,21.658730,3.388889,8.349206
11,High_minus_Low_L1,17.765873,9.849206,9.205026
22,TRANGE_L1,23.626984,7.396825,10.341270
0,ATR_L3,1.773810,30.250000,10.674603
27,High_minus_Low_L3,28.170635,6.472222,11.547619
24,EMA_W_L1,25.500000,10.777778,12.092593


## Summarization for aforementioned periods - final feature choice

In [77]:
df_all_gb = pd.concat([df1_gb, df2_gb, df3_gb])
df_all_gb = df_all_gb.groupby("feature", as_index=False).mean().sort_values("mean_indexes")
features_left = df_all_gb.loc[df_all_gb.mean_indexes <= 25].feature.tolist()
features_removed = df_all_gb.loc[df_all_gb.mean_indexes > 25].feature.tolist()

In [82]:
print(features_left)

['Volume_L1', 'OBV_L1', 'Volume_L2', 'EMA_W_L1', 'Open_stationary_L3', 'MA_L1', 'EMA_L1', 'MA_2_L3', 'MA_W_L2', 'MA_W_L3', 'Volume_L3', 'ATR_L2', 'OBV_L3', 'EMA_W_L3', 'ADOSC_L3', 'High_minus_Low_L3', 'MA_2_L2', 'STD_2_L1', 'TRANGE_L3', 'ADOSC_L1', 'High_stationary_L3', 'STD_W_L2', 'STD_L3', 'Low_stationary_L3', 'STD_L2', 'Low_stationary_L1', 'STD_2_L3']


In [83]:
print(features_removed)

['ATR_L3', 'STD_W_L3', 'OBV_L2', 'MA_W_L1', 'High_stationary_L1', 'EMA_W_L2', 'High_minus_Low_L1', 'STD_W_L1', 'day_of_year', 'MA_2_L1', 'TRANGE_L1', 'ATR_L1', 'MA_L3', 'STD_2_L2', 'Close_minus_Open_L1', 'High_minus_Low_L2', 'ADOSC_L2', 'EMA_2_L1', 'EMA_L2', 'Low_stationary_L2', 'Close_stationary_L1', 'STD_L1', 'rr_L1', 'TRANGE_L2', 'Open_stationary_L1', 'MA_L2', 'Close_stationary_L2', 'week', 'rr_L2', 'Open_stationary_L2', 'rr_L3', 'Close_minus_Open_L2', 'High_stationary_L2', 'Close_stationary_L3', 'EMA_2_L3', 'Close_minus_Open_L3', 'EMA_L3', 'EMA_2_L2', 'day_of_week', 'quarter']


My own adjustment based on experience and EDA analysis

In [84]:
features_left.extend(["rr_L1", "rr_L2", "rr_L3"])

In [86]:
print(features_left)

['Volume_L1', 'OBV_L1', 'Volume_L2', 'EMA_W_L1', 'Open_stationary_L3', 'MA_L1', 'EMA_L1', 'MA_2_L3', 'MA_W_L2', 'MA_W_L3', 'Volume_L3', 'ATR_L2', 'OBV_L3', 'EMA_W_L3', 'ADOSC_L3', 'High_minus_Low_L3', 'MA_2_L2', 'STD_2_L1', 'TRANGE_L3', 'ADOSC_L1', 'High_stationary_L3', 'STD_W_L2', 'STD_L3', 'Low_stationary_L3', 'STD_L2', 'Low_stationary_L1', 'STD_2_L3', 'rr_L1', 'rr_L2', 'rr_L3']


In [104]:
variables_left = features_left
variables_left.insert(0, "rr")

# Saving final dataset

In [107]:
df.dropna(inplace=True)
df = df[variables_left]
df.to_parquet("../dataset/spx_ngboost_final_dataset.parquet")